In [ ]:
# maayanlab.cloud/Enrichr/
# > add background > clear > izq: significativos; dcha: todos (pueden repetirse) > GO biological process. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

In [ ]:
#Abro el .xlsx y selecciono las hojas, filas y columnas que me interesen. En este caso la de p-péptidos
file=pd.ExcelFile('Quantification_TMT-phospho_4conditions-mouse.xlsx')
#cargo la hoja de p-péptidos
df_bg_sheet=file.parse('Peptide-groups_phosho_FDR<1%')
df_sheet1=file.parse('Pep_dif-phos-5%_CreAKI_CreCtrl')
df_sheet2=file.parse('Pep_dif-phos-5%_NapiAKI_NapCtrl')
df_sheet3=file.parse('Pep_dif-phos-5%_CreAKI_NapiCtrl')
df_sheet4=file.parse('Pep_dif-phos-5%_NapiAKI_CreAKI')
df_sheet5=file.parse('Pep_dif-phos-5%_NapiAKI_CreCtrl')
df_sheet6=file.parse('Pep_dif-phos-5%_NapiCtr_CreCtrl')
df_sheet_list=[df_sheet1,df_sheet2,df_sheet3,df_sheet4,df_sheet5,df_sheet6]

In [ ]:
#Función para obtener la lista de nombres de los genes
def get_gene_list(df_sheet):
    descripts_list=df_sheet['Master Protein Descriptions'].tolist()
    gene_names=[]
    descript_count=0
    n_i_count=0
    n_i_list=[]
    for descript in descripts_list:
        if isinstance(descript, float):
            n_i="Not identified"
            descript_count=descript_count+1
            n_i_count=n_i_count+1
            n_i_list.append(n_i)
            continue
        #print(descript)
        elif "GN=" not in descript:
            #n_i="not identified"
            #gene_names.append(n_i)
            descript_count=descript_count+1
            n_i_count=n_i_count+1
            if " OS" in descript:
                n_i_name=descript.split(" OS")[0]
                n_i_list.append(n_i_name)
        else:
            gene=descript.split("GN=")[1].split(' ')[0]
            if gene not in gene_names:
                gene_names.append(gene)
                descript_count=descript_count+1
    n_i_perc=round(n_i_count*100/(descript_count),2)
    gene_count=descript_count-n_i_count
    #print(gene_names)
    #print(n_i_perc)
    #print(descript_count)
    #print(n_i_list)
    return gene_names, gene_count, descript_count, n_i_count, n_i_perc, n_i_list
gene_names, gene_count, descript_count, n_i_count, n_i_perc, n_i_list=get_gene_list(df_sheet1)
print(gene_names)
print(descript_count)
print(n_i_count)
print(gene_count)
print(str(n_i_perc)+'%')
print(n_i_list)
#file=open("genes_list.txt", "w")
#for gene in gene_names:
#    file.write(gene+'\n')
#file.close()

In [ ]:
#Saco la lista de genes "background" y la paso a un .txt, con un gen en cada línea
gene_names, gene_count, descript_count, n_i_count, n_i_perc, n_i_list=get_gene_list(df_bg_sheet)
bg_gene_count=len(gene_names)
print("Parsed genes: "+str(descript_count))
print("Gene list:")
print(gene_names)
print("Number of genes in list: "+str(len(gene_names)))
print("\n"+"Non-identified genes: "+str(n_i_count)+" ("+str(n_i_perc)+'%'+")")
print(n_i_list)
file=open("background_list.txt",'w')
for gene in gene_names:
    file.write(gene+'\n')
file.close()

In [ ]:
gene_count_dict={} #creo un diccionario de listas de genes para ir guardando el número de genes que tienen
#Saco la lista de genes de cada hoja y la paso a un .txt propio de cada una
for i,sheet in enumerate(df_sheet_list):
    gene_names, gene_count, descript_count, n_i_count, n_i_perc, n_i_list=get_gene_list(sheet)
    gene_count_dict[f"gene_count_{i}"]=len(gene_names) #gene_count_dict["gene_count_0"] va a contener el número de genes en la primera lista (la 0) y así puedo usar ese número más tarde para el fold enrichment
    print("Parsed genes: "+str(descript_count))
    print("Gene list:")
    print(gene_names)
    print("Number of genes in list: "+str(len(gene_names)))
    print("Non-identified genes: "+str(n_i_count)+" ("+str(n_i_perc)+'%'+")")
    print(n_i_list)
    print("\n")
    for gene in gene_names:
        file=open("gene list "+str(i)+".txt",'w')
        for gene in gene_names:
            file.write(gene+'\n')
        file.close()
#print(gene_count_dict["gene_count_1"])

In [ ]:
#La tabla obtenida de Enrichr tiene el siguiente formato:
#Term	Overlap	P-value	Adjusted P-value	Old P-value	Old Adjusted P-value	Odds Ratio	Combined Score	Genes [1ª línea]
#Regulation Of Long-Term Synaptic Depression (GO:1900452)	2/2	0.002875624719213879	0.40992235900376406	0	0	Infinity	Infinity	MAPT;SHANK3 [resto de líneas]
#Las "columnas" están separadas por "\t". Nos interesa las columnas 0, 1, 2 y 8 (GO, overlap, p-value, genes)
#Quiero crean un df con la info modifcada: overlap pasado a fold enrichment, -log10 de p-value y añadiendo el número de genes
#Fold enrichment = (nº de genes de la lista con un GO / nº de genes en la lista) / (nº de genes del background con el mismo GO / nº de genes en background). Enrichr solo nos da en el overlap los números con un GO, tenemos que dividir nosotros entre los números totales en las listas 
import math #para hacer después el -log10(p-value)
def get_GO_df(GO_file):
    #Creo un df con las siguientes columnas:
    col_names=["GO","Fold enrichment","-log10(p-value)","Genes","Number of genes"]
    GO_df=pd.DataFrame(columns=col_names)
    #Inicio las listas con las columnas que voy a añadir después al df
    GO_col=[]
    FE_col=[]
    p_col=[]
    gene_col=[]
    n_genes_col=[]
    file_number=f"{GO_file}"[-5] #quiero saber en qué número de archivo estoy: f"{GO_file}" para GO gene list 0.txt es "GO gene list 0.txt"; la última posición de ese str es la -1, pues la -5 es el número antes de ".txt"
    lines_list=open(GO_file).readlines() #creo una lista, cada elemento de la lista es una línea del archivo, ej.: Regulation Of Long-Term Synaptic Depression (GO:1900452)\t2/2\t0.002875624719213879\t0.40992235900376406\t0\t0\tInfinity\tInfinity\tMAPT;SHANK3\n
    lines_list=lines_list[1:] #quito la primera línea [la 0] (donde vienen los nombres de las "columnas")
    for line in lines_list:
        line=line.replace("\n","") #quito el "\n" al final de cada línea
        line_elements=line.split("\t") #creo una lista con los elementos de cada línea, separando por "\t"
        p_val=float(line_elements[2]) #extraigo el p-value que aparece en cada línea. Si tenemos decimales no se puede usar int() para convertir una str a número, hay que usar float()
        if p_val<=0.01: #solo trabajo con las líneas con un p-value < 0,05
            p_val=-math.log10(p_val) #paso el p-value a -log10. -log10(0,05) = 1,3, por lo que los valores significativos son mayores a 1,3
            p_col.append(p_val) #añado el p-value a la lista de p-values
            GO_col.append(line_elements[0]) #añado el GO
            overlap_values=line_elements[1].split("/") #el overlap viene como "(número de genes en la lista que pertenecen al GO)/(número de genes del background que pertenecen al GO)", así que separo ambos (ej: 2/3)
            fold_enr_num=int(overlap_values[0])/(gene_count_dict[f"gene_count_{file_number}"]) #numerador para calcular el FE: divido el número de genes en la lista que pertenecen al GO entre el número total de genes en la lista (ej: 2/98)
            fold_enr_denom=int(overlap_values[1])/(bg_gene_count) #denominador para calcular el FE: divido el número de genes del background que pertenecen al GO entre el número total de genes en el background (ej: 3/1800). 
            fold_enr=fold_enr_num/fold_enr_denom
            FE_col.append(fold_enr) #añado el fold enrichment a la lista de FEs
            genes_in_line=line_elements[8].split(";") #creo una lista con los genes en esa línea (están separados por ";")
            n_genes_col.append(len(genes_in_line)) #la longitud de la lista es el número de genes que tengo en esa línea
            new_genes_in_line=''#inicio un objeto para añadir aquí los genes después de ponerlos en el formato que quiero (solo la primera letra en mayúscula y separados por comas)
            for gene in genes_in_line:
                gene=gene.capitalize() #los genes vienen en mayúsculas, solo quiero que la primera letra sea mayúscula: .capitalize()
                new_genes_in_line=new_genes_in_line+gene+"," #añado el gen al objeto de genes y una coma para separarlo del siguiente 
            new_genes_in_line=new_genes_in_line[:-1] #quito el último carácter del objeto de genes (que es la coma añadida con el último gen)
            gene_col.append(new_genes_in_line) #añado el objeto con genes a la lista de genes
        else:
            break
    #Añado cada lista a su correspondiente columna en el df
    cols_to_add=[GO_col,FE_col,p_col,gene_col,n_genes_col] #hago una lista con las lista que he creado, en el mismo orden que las columnas del df
    for i,col in enumerate(col_names): #con i y enumerate empiezo con el nombre de la primera columna ("GO") e i=0. Con la siguiente columna (Fold enrichment), i pasa a ser 1
        GO_df[f'{col}']=cols_to_add[i] #añado a la columna que se esté iterando la lista correspondiente que he creado antes (ej. GO_df["GO"]=GO_col, es decir, añado a la columna "GO" en el GO_df la lista GO_col (que está en la posición i=0 de cols_to add)
    return GO_df
get_GO_df("GO gene list 0.txt")

In [ ]:
#Genero una lista con los nombres de los archivos txt de Enrichr
GO_txt_files_list=[]
for i in range(len(df_sheet_list)): #range(len) va de 0 hasta la longitud de la lista *-1*
    GO_txt_files_list.append(f"GO gene list {i}.txt")
GO_txt_files_list
#Creo un diccionario de GO_dfs
GO_dfs={}
for i,file in enumerate(GO_txt_files_list):
    GO_dfs[f"GO_df_{i}"]=get_GO_df(file)
GO_dfs["GO_df_5"]
GO_df_0=GO_dfs["GO_df_0"]

In [ ]:
#Genero un bubble chart para cada df
for i in range(len(GO_txt_files_list)):
    df=GO_dfs[f"GO_df_{i}"] #cada df guardado en el diccionario lo guardo como df para representarlo
    #Creo el bubble chart con plt
    plt.figure(figsize=(12.5,10))  #(ancho, alto)
    scatter = plt.scatter(df['Fold enrichment'], df['-log10(p-value)'], s=df['Number of genes']*200, color="purple", alpha=0.5) #(valores eje x, valores eje y, tamaño de los círculos, transparencia).
    plt.xlabel('Fold enrichment')
    plt.ylabel('-log$_{10}$(p-value)') #$_{10}$: $$ para usar LaTeX; "_" para subscript
    x_lim=math.ceil(max(df['Fold enrichment']))
    y_lim=math.ceil(max(df['-log10(p-value)']))
    plt.xlim(0,x_lim+.5) #límite eje x
    plt.ylim(1.33,y_lim-.3) #límite eje y
    #Si se ven colores más oscuros quiere decir que hay solapamientos
    """for j in range(len(df)):
        x = df['Fold enrichment'].iloc[j]
        y = df['-log10(p-value)'].iloc[j]
        #plt.plot([x, x], [y, y+0.03], color='k', linewidth=0.2)  # Adjust the length and thickness here
        plt.text(x, y+0.035, df['GO'].iloc[j], ha='center',fontsize=4)  # Adjust the position of the text here"""
    plt.savefig(f"GO_bio_process_{i}.svg")
    plt.show()
#Genero un plot con la leyenda
plt.figure(figsize=(1,3.5)) #(ancho, alto)
size = np.array([2, 4, 6, 8])*200 #tamaño de los círculos (4 círculos)
#Posición x e y de los círculos
x = np.ones(4)  #mismo valor de x para los 4 círculos: array de cuatro "unos"
y = np.linspace(3.2,.8, 4)  #valores con el mismo espacio entre ellos (primer valor, último valor, nº de valores)
#Creo el bubble chart
plt.scatter(x, y, s=size, color="purple", alpha=0.5)
#Añado labels a cada burbuja
for i in range(len(x)):
    plt.text(x[i]+.55, y[i], str(size[i]//200), ha='center', va='center', color='black') #x[i] + .55 mueve a la derecha los labels
#Límites de los ejes
plt.xlim(0.5, 1.5)  # eje x
plt.ylim(0, 3.5)  # eje y
plt.axis("off") #quito los ejes
plt.title("Number of genes", loc='center') #añado un título
plt.savefig("GO_legend.svg")
plt.show()